In [ ]:
from spf.dataset.fake_dataset import create_fake_dataset, fake_yaml
import tempfile


tmpdir = tempfile.TemporaryDirectory()
ds_fn = f"{tmpdir.name}/test_circle"
create_fake_dataset(yaml_config_str=fake_yaml, filename=ds_fn, orbits=2, n=800)

In [ ]:
import os
import spf

# this contains the empirical distribution mapping observed phase difference
# to the data distribution of theta (ground truth angle between receiver and transmitter)
empirical_data_fn = os.path.join(
    os.path.dirname(spf.__file__), "../empirical_dists/full.pkl"
)

In [ ]:
# create one cache directory for non temp files
# when you re-run this line you need to rebuild the cache
tmp_precompute_cache = tempfile.TemporaryDirectory()
precompute_cache_dir = tmp_precompute_cache.name

In [ ]:
from spf.dataset.spf_dataset import v5spfdataset_manager
from spf.filters.particle_dualradio_filter import plot_single_theta_dual_radio
from spf.filters.run_filters_on_data import run_PF_single_theta_dual_radio


# When false consider this complete file that we can fully read
# instead of a file that we will partially read and wait for further
# data on (live)
temp_file = False

with v5spfdataset_manager(
    ds_fn,
    precompute_cache=precompute_cache_dir,
    nthetas=65,
    skip_fields=set(["signal_matrix"]),
    empirical_data_fn=empirical_data_fn,
    paired=True,
    ignore_qc=True,
    gpu=True,  # use GPU in segmentation (pre-processing)
    temp_file=temp_file,
    temp_file_suffix="",
    segment_if_not_exist=not temp_file,
) as ds:
    args = {
        "ds": ds,
        "N": 1024 * 4,
        "theta_err": 0.01,
        "theta_dot_err": 0.01,
    }
    result = run_PF_single_theta_dual_radio(**args)
    plot_single_theta_dual_radio(ds)

In [ ]:
plot_single_theta_dual_radio(ds)

In [ ]:
import importlib
import importlib.resources

importlib.resources.files("spf.empirical_dists")._paths[0]

In [ ]:
import pickle

pickle.load(open(empirical_data_fn, "rb")).keys()